In [1]:
from pandas import DataFrame, read_csv
import pandas as pd
import numpy as np
import xlrd

In [2]:
def getColor(book, sheet, row, col):
    xfx = sheet.cell_xf_index(row, col)
    xf = book.xf_list[xfx]
    bgx = xf.background.pattern_colour_index
    color_code = None
    if bgx == 9:
        color_code = 0
    elif bgx == 42:
        color_code = 1
    elif bgx == 10:
        color_code = -1
    #pattern_colour = book.colour_map[bgx]
    return color_code

In [3]:
fin = 'data/jfreechart-1.0.10-ccfinder.xls'
df = pd.read_excel(fin)
fin_analyzed = 'data/jfreechart-1.0.10-ccfinder-analyzed.xls'
book = xlrd.open_workbook(fin_analyzed, formatting_info=True)
sheet = book.sheet_by_index(0)

In [4]:
df.dtypes

Clone Group ID           int64
Source Folder           object
Package                 object
Class                   object
Method                  object
Method Signature        object
Start Line               int64
End Line                 int64
Start Offset             int64
End Offset               int64
#PDG Nodes             float64
#Statements            float64
Line Coverage (%)      float64
Clone Group Size       float64
Clone Group Info        object
Connected              float64
Clone Pair Location     object
#Refactorable Pairs    float64
Details                float64
dtype: object

In [5]:
bgx = []
col = 18
for i in range(1, sheet.nrows):
    bgx.append(getColor(book, sheet, i, col))
colors = pd.Series(np.array(bgx), index=df.index)
df['Colors'] = colors

In [6]:
grouped = df.groupby('Clone Group ID')
selected = grouped.filter(lambda group: 
               len(group) == 2 
               and group['Method'].isnull().sum() == 0 
               and group['Method Signature'].isnull().sum() == 0 
               and (group['Source Folder'] != 'tests').sum() == 0 
               and (group['Method'] == 'suite').sum() == 0)

In [7]:
fout = 'output/jfreechart-1.0.10-selected.xls'
writer = pd.ExcelWriter(fout)
selected.to_excel(excel_writer = writer, sheet_name = 'Sheet1', index = False)
writer.save()
#selected.to_csv(fout)
#selected.to_excel(fout)